In [ ]:
from pyspark.sql import functions as F


In [ ]:
stock_price_date = dbutils.widgets.get("stock_price_date")
directory_path = dbutils.widgets.get("directory_path")


In [ ]:
stock_price_df = spark.read.parquet(f"{directory_path}/stock_price_{stock_price_date}")


In [ ]:
stock_price_df = (stock_price_df.withColumnRenamed("Adj Close", "Adj_Close")
                  .withColumn("metadata", 
                     F.create_map(
                        F.lit("pipeline_id"), F.lit("pipeline_id"),
                        F.lit("run_id"), F.lit("run_id"),
                        F.lit("task_id"), F.lit("task_id"),
                        F.lit("processed_timestamp"), F.lit("processed_timestamp")
                  )))


In [ ]:
if spark.catalog.tableExists("`yfinance-dev`.bronze.stock_price"):
    spark.sql(f'DELETE FROM `yfinance-dev`.bronze.stock_price WHERE date = "{stock_price_date}"')


DataFrame[num_affected_rows: bigint]

In [ ]:
display(stock_price_df)


In [45]:
(stock_price_df.write.mode("append")
 .saveAsTable("`yfinance-dev`.bronze.stock_price")
)
